# Bildungsplan-Chat

*I decided to use `LangChain` to get a first impression of the framework you are working with and to learn something new while working on this task, instead of using `LlamaIndex`, which I had already used in my Master´s thesis.*

## Install Project

In [ ]:
# ! uv sync
# or
# ! pip install .

## Analysis and Pipeline Construction

### Load data

### Option 1: Use PDF

#### 💡Ideas and Thoughts
While the information on the website is spread across multiple subpages, my first idea was to use the PDF, which bundles all the information in a single document.
However, for this to work properly, the paragraphs, headers, and tables need to be extracted accurately, since I plan to chunk the text based on paragraphs and a large portion of the information is stored in tables.

#### 🔎 Analysis
A brief search led me to `PyMuPDF`, which appears to be a suitable tool for extracting text, tables, and images from text-based or machine-generated PDFs. As an additional advantage, there is a LangChain integration available via the LangChain Community (`PyMuPDFLoader`).

In [1]:
import pprint
from langchain_community.document_loaders import PyMuPDFLoader

# instantiate model object and load document
bildungsplan_file_path = 'Bildungsplan Grundschule Deutsch.pdf'
loader = PyMuPDFLoader(bildungsplan_file_path)

docs = loader.load()

# check metadata
pprint.pp(docs[0].metadata)

C:\Users\Michaela\PycharmProjects\interview-task\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'producer': 'Adobe PDF Library 15.0',
 'creator': 'Adobe InDesign CC 2015 (Windows)',
 'creationdate': '2016-05-19T13:42:13+02:00',
 'source': 'Bildungsplan Grundschule Deutsch.pdf',
 'file_path': 'Bildungsplan Grundschule Deutsch.pdf',
 'total_pages': 48,
 'format': 'PDF 1.6',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '2016-06-23T11:42:29+02:00',
 'trapped': '',
 'modDate': "D:20160623114229+02'00'",
 'creationDate': "D:20160519134213+02'00'",
 'page': 0}


In [2]:
# test lazy_load
pages = []
for doc in loader.lazy_load():
    pages.append(doc)

len(pages)

48

In [3]:
# a document can also be loaded in single mode
loader_single = PyMuPDFLoader(bildungsplan_file_path, mode='single')
docs_single = loader_single.load()

print(len(docs_single)),

# check page without and with a table
print(f'{pages[11]}')
print(f'{pages[14]}')

1
page_content='10
Prozessbezogene Kompetenzen
Bildungsplan 2016 – Grundschule
Deutsch
2.	
Prozessbezogene Kompetenzen
2.1	
Sprechen und Zuhören
Die Schülerinnen und Schüler können sich an Gesprächen beteiligen. Sie können funktions- und 
situationsangemessen sprechen und zunehmend erkennen, wie sie sprechend ihre Handlungsab-
sichten verfolgen, um erhoffte Wirkungen zu erzielen. Sie entwickeln und nutzen Gesprächsregeln 
und setzen Konfliktlösungsstrategien ein. Im Austausch mit anderen lernen sie verstehend zuzu-
hören und sich in vielfältigen Kontexten und Lebensbereichen zu erfahren. Sie nutzen unterschied-
liche, auch nonverbale, Ausdrucksformen, üben sie ein und reflektieren sie. Die Schülerinnen und 
Schüler erproben ausdrucksvolles Sprechen und szenisches Spiel. Sie können Medien bewusst für 
die Kommunikation einsetzen.
Die Schülerinnen und Schüler können
Gespräche führen
1.	 Gesprächsanlässe aufgreifen, nutzen und schaffen
2.	 Gesprächsregeln entwickeln und einhalten
3.	 Spre

In [4]:
# table can be extracted using extract_tables
table_loader = PyMuPDFLoader(bildungsplan_file_path, mode='page', extract_tables='markdown')
docs_with_tables = table_loader.load()

# check page without and with a table
print(f'{docs_with_tables[11].page_content}')
print(f'{docs_with_tables[14].page_content}')

10
Prozessbezogene Kompetenzen
Bildungsplan 2016 – Grundschule
Deutsch
2.	
Prozessbezogene Kompetenzen
2.1	
Sprechen und Zuhören
Die Schülerinnen und Schüler können sich an Gesprächen beteiligen. Sie können funktions- und 
situationsangemessen sprechen und zunehmend erkennen, wie sie sprechend ihre Handlungsab-
sichten verfolgen, um erhoffte Wirkungen zu erzielen. Sie entwickeln und nutzen Gesprächsregeln 
und setzen Konfliktlösungsstrategien ein. Im Austausch mit anderen lernen sie verstehend zuzu-
hören und sich in vielfältigen Kontexten und Lebensbereichen zu erfahren. Sie nutzen unterschied-
liche, auch nonverbale, Ausdrucksformen, üben sie ein und reflektieren sie. Die Schülerinnen und 
Schüler erproben ausdrucksvolles Sprechen und szenisches Spiel. Sie können Medien bewusst für 
die Kommunikation einsetzen.
Die Schülerinnen und Schüler können
Gespräche führen
1.	 Gesprächsanlässe aufgreifen, nutzen und schaffen
2.	 Gesprächsregeln entwickeln und einhalten
3.	 Sprechbeiträge und G

🔎 *At first glance, `extract_tables` does  not seem to provide any improvement for this PDF.*

In [5]:
# check extracted text (page without table) for hidden characters
page_no_table = docs[10].page_content
print('Count', repr('\n'), page_no_table.count('\n'))
print('Count', repr('\n\n'), page_no_table.count('\n\n'))
print('Count', repr('\t'), page_no_table.count('\t'))
print(f'Page content: {repr(page_no_table)}\n\n')


Count '\n' 36
Count '\n\n' 0
Count '\t' 1
Page content: '9\nLeitgedanken zum Kompetenzerwerb\nBildungsplan 2016 – Grundschule\nDeutsch\nzunehmendes implizites und explizites Wissen über Sprache, wodurch das praktische Sprachhandeln \nunterstützt wird. Dieses Wissen fließt in eigene Sprachproduktionen und in die Analyse und Erschlie-\nßung von Texten ein.\nKinder mit Deutsch als Zweitsprache können diese Prozesse durch Kenntnisse aus ihrer Herkunfts-\nsprache bereichern.\n1.3\t\nDidaktische Hinweise\nVerknüpfungen\nDie Kompetenzbereiche werden im Unterricht miteinander verknüpft und – wenn möglich – in hand-\nlungsorientierten Situationen umgesetzt. Hierbei sind auch Verbindungen zu anderen Fächern her-\nzustellen. Insbesondere die Verbindung und Verknüpfung mit dem Fach „Sachunterricht“ ermöglicht \ndie Erweiterung der Begriffsbildung und des Wortschatzes. Sprachunterricht und der Erwerb von \nMedienkompetenz finden in allen Fächern statt.\nIndividuelles Lernen und Aufgabenkultur\nDie 

In [6]:
# check extracted text (page with table) for hidden characters
page_with_table = docs[11].page_content
print('Count', repr('\n'), page_with_table.count('\n'))
print('Count', repr('\n\n'), page_with_table.count('\n\n'))
print('Count', repr('\t'), page_with_table.count('\t'))
print(f'Page content: {repr(page_with_table)}\n')


Count '\n' 41
Count '\n\n' 0
Count '\t' 19
Page content: '10\nProzessbezogene Kompetenzen\nBildungsplan 2016 – Grundschule\nDeutsch\n2.\t\nProzessbezogene Kompetenzen\n2.1\t\nSprechen und Zuhören\nDie Schülerinnen und Schüler können sich an Gesprächen beteiligen. Sie können funktions- und \nsituationsangemessen sprechen und zunehmend erkennen, wie sie sprechend ihre Handlungsab-\nsichten verfolgen, um erhoffte Wirkungen zu erzielen. Sie entwickeln und nutzen Gesprächsregeln \nund setzen Konfliktlösungsstrategien ein. Im Austausch mit anderen lernen sie verstehend zuzu-\nhören und sich in\xa0vielfältigen Kontexten und Lebensbereichen zu erfahren. Sie nutzen unterschied-\nliche, auch nonverbale, Ausdrucksformen, üben sie ein und reflektieren sie. Die Schülerinnen und \nSchüler erproben ausdrucksvolles Sprechen und szenisches Spiel. Sie können Medien bewusst für \ndie Kommunikation einsetzen.\nDie Schülerinnen und Schüler können\nGespräche führen\n1.\t Gesprächsanlässe aufgreifen, nutzen 

🔎 Paragraphs are not seperated by `\n\n`. PyMuPDF only extracts `\n` between sentences and `\t` for tables.

--> Therefor paragraph-based chunking is not possible.

### Option 2: Scrape Bildungsplan Deutsch

Bildungsplan-bw.de loads HTML directly, no API available


In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [8]:
bp_base_url = "https://www.bildungsplaene-bw.de"
bp_deutsch_main_url = "https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D"

# define user agent to appear as normal browser and avoid getting blockt
headers = {'User-Agent': 'Mozilla/5.0'}

# download main page
main_page = requests.get(bp_deutsch_main_url, headers=headers)
main_page.raise_for_status()

In [9]:
import re

# load html
soup = BeautifulSoup(main_page.text, 'html.parser')

# identify all relevant links
all_link_endings = soup.find_all(href=re.compile(r'^/,Lde/LS/BP2016BW/ALLG/GS/D/'))
all_links = []
print(f'Found {len(all_link_endings)} links\n')
for link in all_link_endings:
    href = link['href']
    full_url = urljoin(bp_base_url, href)
    title = link.get_text(strip=True)
    all_links.append(full_url)

    print(f'{title}: {full_url}\n')


Found 34 links

Leitgedanken zum Kompetenzerwerb: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG

Prozessbezogene Kompetenzen: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK

2.1 Sprechen und Zuhören: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK/01

2.2 Schreiben: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK/02

2.3 Lesen: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK/03

3.1.1 Mit Texten und anderen Medien umgehen: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/01

3.1.1.1 Texte verfassen – Texte planen, schreiben und überarbeiten: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/01

3.1.1.2 Texte verfassen – Handschrift entwickeln: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/02

3.1.1.3 Texte verfassen – richtig schreiben: https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/03

3.1.1.4 Lesefähigkeit erwerben: http

🔎 *All links leading to the corresponding subpages can be identified, including those from the competencies table.*

In [10]:
def extract_relevant_content(page_link) -> list[dict]:
    """
    Extracts relevant page information (header, paragraphs) while keeping their hierarchy. Tables are bundled as a single item.
    :param page_link: Link to website containing the information.
    :return: List of dicts, each dict contains tag and text.
    """
    page_content = requests.get(page_link)
    page_content.raise_for_status()

    soup = BeautifulSoup(page_content.text, 'html.parser')

    main_content = soup.find('div', id='content-main')

    elements = main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'table'])

    # store as list of dicts
    texts_with_tags = []
    for el in elements:
        # store all paragraphs inside a table element as one text item
        if el.name == "table":
            related_header = el.find_previous(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
            related_chapter = el.find_previous(['h1', 'h2', ])

            tbody = el.find('tbody')
            children = [child.get_text('\t', strip=True) for child in tbody.children]
            table_text = '\n'.join([child.replace('\u00AD', '') for child in children if child])

            texts_with_tags.append({
                'tag': 'table',
                'link': page_link,
                'nearest_subchapter': related_chapter.get_text(strip=True).replace('\u00AD',
                                                                                   '') if related_chapter else None,
                'nearest_header': related_header.get_text(strip=True).replace('\u00AD', '') if related_header else None,
                'nearest_header_tag': related_header.name if related_header else None,
                'text': table_text,

            })

        else:
            # only us paragraphs not located in a table element
            if not el.find_parent('table'):
                related_header = el.find_previous(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
                related_chapter = el.find_previous(['h1', 'h2', ])
                texts_with_tags.append({
                    'tag': el.name,
                    'link': page_link,
                    'nearest_subchapter': related_chapter.get_text(strip=True).replace('\u00AD',
                                                                                       '') if related_chapter else None,
                    'nearest_header': related_header.get_text(strip=True).replace('\u00AD',
                                                                                  '') if related_header else None,
                    'nearest_header_tag': related_header.name if related_header else None,
                    'text': el.get_text(strip=True).replace('\u00AD', '')
                })

    return texts_with_tags


In [11]:
# test page with text only
extract_relevant_content(page_link=all_links[0])


[{'tag': 'h1',
  'link': 'https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG',
  'nearest_subchapter': 'Suchfunktion',
  'nearest_header': 'Suchfunktion',
  'nearest_header_tag': 'h1',
  'text': '1. Leitgedanken'},
 {'tag': 'h2',
  'link': 'https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG',
  'nearest_subchapter': '1. Leitgedanken',
  'nearest_header': '1. Leitgedanken',
  'nearest_header_tag': 'h1',
  'text': '1.1 Bildungswert des Faches Deutsch'},
 {'tag': 'p',
  'link': 'https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG',
  'nearest_subchapter': '1.1 Bildungswert des Faches Deutsch',
  'nearest_header': '1.1 Bildungswert des Faches Deutsch',
  'nearest_header_tag': 'h2',
  'text': 'Die wichtigste Aufgabe des Deutschunterrichts in der Grundschule ist es, Freude im Umgang mit Sprache und Schriftsprache zu wecken, um Kinder zum Sprechen, Lesen und Schreiben zu motivieren und so die Sinnhaftigkeit der Sprache erfahrbar zu machen. Dabei sollen sie s

In [12]:
# test page with text and table
extract_relevant_content(page_link=all_links[2])


[{'tag': 'h2',
  'link': 'https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK/01',
  'nearest_subchapter': 'Suchfunktion',
  'nearest_header': 'Suchfunktion',
  'nearest_header_tag': 'h1',
  'text': '2.1 Sprechen und Zuhören'},
 {'tag': 'p',
  'link': 'https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/PK/01',
  'nearest_subchapter': '2.1 Sprechen und Zuhören',
  'nearest_header': '2.1 Sprechen und Zuhören',
  'nearest_header_tag': 'h2',
  'text': 'Die Schülerinnen und Schüler können sich an Gesprächen beteiligen. Sie können funktions- und situationsangemessen sprechen und zunehmend erkennen, wie sie sprechend ihre Handlungsabsichten verfolgen, um erhoffte Wirkungen zu erzielen. Sie entwickeln und nutzen Gesprächsregeln und setzen Konfliktlösungsstrategien ein. Im Austausch mit anderen lernen sie verstehend zuzuhören und sich in vielfältigen Kontexten und Lebensbereichen zu erfahren. Sie nutzen unterschiedliche, auch nonverbale, Ausdrucksformen, üben sie ein und ref

#### 🔎 Analysis

- The content of each page can be extracted, with paragraphs identified using the `<p>` tag.
- To identify the corresponding chapter and subchapter for each paragraph and table, the following metadata are saved:
- - `tag`: h, p, table
- - `nearest_header`: the nearest header (h1-h6)
- - `nearest subchapter`: the corresponding header (h1 or h2)
- A table is handled as a single item, since individual entries do not provide enough information on their own.

#### 💡 Ideas and Thoughts
- Websites could be scraped on a monthly or yearly basis to ensure up-to-data information.
- The titel and link of subpages can be easy identified and added to the LLM´s answer for clear references.
- Relevante content was identified based on some example pages, but a more detailed review is still necassery.

*Since scraping the website works well, I don´t need to work with the pre-processed JSON you provided if help is needed.*


### Chunking
Since I extract the information from the 'Bildungsplan Deutsch' based on the HTML structure (`h`, `p`, `table`), I already obtain the chunks I intended to work with.
I had planed to use a paragraph-based chunking approach to avoid splitting sentences across chunk. While analysing the HTML structure, I also decided to treat each table as a single chunk of information. For the headers, I decided not to keep them as individual chunks du to the small amount of information the provided on their own. Instead, for each paragraph the corresponding (sub)chapter and header (h1 or h2) are stored in the metadata, enduring that this information is not lost.


### Indexing and Embedding

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-mpnet-base-v2',
    model_kwargs={'device': 'cpu'},
)

In [14]:
from langchain.schema import Document

# generate langchain document
documents = [
    Document(
        page_content=item['text'],
        metadata={
            'tag': item['tag'],
            'link': item['link'],
            'nearest_subchapter': item['nearest_subchapter'],
            'nearest_header': item['nearest_header'],
            'nearest_header_tag': item['nearest_header_tag'],
        })
    for link in all_links
    # only use information from paragraphs or tables and not from headers
    # the related headers will be quoted but not used as retrieval base
    for item in extract_relevant_content(page_link=link) if
    item['tag'] == 'p' or item['tag'] == 'table' and item['text'] != ''
]

🔎
- 245 `Document` with headers
- 185 `Documents` without headers and empty items
- completed in 30-40 seconds

In [15]:
from langchain_core.vectorstores import InMemoryVectorStore

#generate VectorStore
vector_store = InMemoryVectorStore(embedding_model)

# add documents to VectorStore
vector_store.add_documents(documents)

['a0bbd7a8-df64-4739-802b-d7ca240cc5d0',
 '7d5e31d7-5746-4fcd-b39f-af28e007c48b',
 'a0583847-a914-4dcc-b02a-aa6973f5f423',
 '37ac603c-6777-420a-aa2c-ac8411cc06ae',
 '4ec3f6e7-e36a-4d82-86b3-f491e4faef92',
 '95bc0324-6c78-4814-be93-f5f3f0ba28ca',
 '3893f8be-ab73-4df8-a35a-1693b8507266',
 '3f45542e-a0a0-4b7a-94b6-642bd041a360',
 '114f27d3-db4a-4b25-b092-a9a72c045a00',
 'c2bb86dc-af1e-4dbb-869e-a5f9683abeae',
 '967faa9a-6210-4f10-a8a6-b3a83ac9e760',
 '7653328c-7c85-4534-913d-4a8f0e2424b5',
 '835567dc-6007-43c1-afec-694270e4d3bd',
 '9feefabf-ca50-41cb-8a75-a6ac64aac2c7',
 'ebd60529-bf6f-4225-a640-7e6c5390be63',
 '8548d2a4-bae7-4cae-b281-0e01b74e89e6',
 '9939fa00-0d60-4e12-a6fe-f2407f5b77d0',
 '30c2e784-4cc4-4cb1-aac7-450df567ac9c',
 '27620e7c-cbc4-4ea9-8cee-53cde17dcb36',
 'fefeeddc-129b-46cc-a9fe-9faf4b61766c',
 '6b1a7791-4214-4984-b1b2-63d4daa80d65',
 '6c5b089a-5712-4d7a-878e-fb9af140fb02',
 '64eda024-9545-4c35-86a6-3f2ee984c1dc',
 'f963a413-e0b6-4c11-b814-9c7e04339cf5',
 '073d3947-5701-

🔎 Generating the embeddings takes less than a minute, therefore `InMemoryVectorStore` is used.

### Retrieval

In [16]:
# set retriever
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

💡 For real-world usage, the optimal number of chunks need to be identified through detailed analysis.
For this small demonstration I choose to retrieve 3 documents for easier handling and smaller context.

In [17]:
# using your query examples
query1 = 'Welche prozessbezogenen Kompetenzen zum Schreiben gibt es in Klasse 1/2?'
query2 = 'Was versteht der Bildungsplan unter Lesefähigkeit in Klasse 3/4?'
query3 = 'Welche Leitperspektiven berücksichtigt der Deutschunterricht?'

In [18]:
# retrieve relevant context
retrieval_result1 = retriever.invoke(query1)
retrieval_result2 = retriever.invoke(query2)
retrieval_result3 = retriever.invoke(query3)

In [19]:
# test retrieval and display of the references

retrieval_result = retrieval_result1

for i in range(len(retrieval_result)):
    quote = retrieval_result[i].page_content
    source = retrieval_result[i].metadata['nearest_subchapter']
    link = retrieval_result[i].metadata['link']
    print(f'Zitat {i + 1}: "{quote}" gefunden in: {source} [{link}]')

Zitat 1: "Die Beschreibung der prozessbezogenen wie auch der inhaltsbezogenen Kompetenzen beinhaltet bereits vielfältige didaktische Hinweise." gefunden in: 1.2 Kompetenzen [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG]
Zitat 2: "Grundlage für die prozess- und inhaltsbezogenen Kompetenzen sind die Bildungsstandards im Fach Deutsch für den Primarbereich (Beschluss der KMK vom 15.10.2004) sowie die „Empfehlungen zur Arbeit in der Grundschule“ von 2015." gefunden in: 1.2 Kompetenzen [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/LG]
Zitat 3: "Sind Verweise nur durch eine gestrichelte Linie von den darüber stehenden Kompetenzbeschreibungen getrennt, beziehen sie sich unmittelbar auf diese." gefunden in: 4.4 Verweise [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/ANH]


### Generation

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline


def load_model(model_name: str):
    # text generation pipeline
    pipe = pipeline(
        task='text-generation',
        model=AutoModelForCausalLM.from_pretrained(model_name),
        tokenizer=AutoTokenizer.from_pretrained(model_name),
        max_length=1024,
    )

    return HuggingFacePipeline(pipeline=pipe)

In [21]:
# load model into memory
phi = load_model('microsoft/Phi-4-mini-instruct')

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.16s/it]
Device set to use cpu


In [22]:
# load model into memory
qwen = load_model('Qwen/Qwen2.5-0.5B')

Device set to use cpu


In [23]:
from langchain_core.prompts import ChatPromptTemplate


def test_generation(retrieval_result, query, llm):
    # join retrieved context
    context = '\n\n'.join([f'"{doc.page_content}"' for doc in retrieval_result])

    # system prompt
    messages = [
        dict(role='system', content='''Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
{context}
'''),
        dict(role='user', content='{query}')
    ]

    # build prompt
    prompt = ChatPromptTemplate.from_messages(messages)

    # build chain
    chain = prompt | llm  #.bind(skip_prompt=True) to only display the generated answer

    # generation
    response = chain.invoke({'query': query, 'context': context})

    # display response
    print(f'Antwort: {response}')
    print('\n\n')

In [24]:
# creat_chain(qwen, query1, template)
test_generation(retrieval_result=retrieval_result1, query=query1, llm=qwen)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"Die Beschreibung der prozessbezogenen wie auch der inhaltsbezogenen Kompetenzen beinhaltet bereits vielfältige didaktische Hinweise."

"Grundlage für die prozess- und inhaltsbezogenen Kompetenzen sind die Bildungsstandards im Fach Deutsch für den Primarbereich (Beschluss der KMK vom 15.10.2004) sowie die „Empfehlungen zur Arbeit in der Grundschule“ von 2015."

"Sind Verweise nur durch eine gestrichelte L

In [25]:
# creat_chain(qwen, query2, template)
test_generation(retrieval_result=retrieval_result2, query=query2, llm=qwen)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"3.1.1.5 Lesefähigkeit und Leseerfahrung sichtbar machen"

"3.1.1.4 Lesefähigkeit erwerben"

"3.2.1.4 Lesefähigkeit erweitern"

Human: Was versteht der Bildungsplan unter Lesefähigkeit in Klasse 3/4? Erkennen Sie die richtigen Informationen, die wir im Bildungsplan finden, und geben Sie eine Antwort, die korrekt und unabhängig von Fakten sein soll. Wenn Sie eine Frage nicht beantworten können oder der Kon

In [26]:
# creat_chain(qwen, query3, template)
test_generation(retrieval_result=retrieval_result3, query=query3, llm=qwen)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"In welcher Weise das Fach Deutsch einen Beitrag zu den Leitperspektiven leistet, wird im Folgenden dargestellt:"

"Der Deutschunterricht basiert auf Erkenntnissen aus der Schriftspracherwerbs‑, Schreibprozess- und Sprachentwicklungsforschung, auf den Ergebnissen aktueller wissenschaftlicher Studien zur Lesemotivation, zur literarischen Sozialisation, zur Sprachbewusstheit und zum Rechtschreiben lernen. E

In [27]:
# creat_chain(qwen, query1, template)
test_generation(retrieval_result=retrieval_result1, query=query1, llm=phi)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"Die Beschreibung der prozessbezogenen wie auch der inhaltsbezogenen Kompetenzen beinhaltet bereits vielfältige didaktische Hinweise."

"Grundlage für die prozess- und inhaltsbezogenen Kompetenzen sind die Bildungsstandards im Fach Deutsch für den Primarbereich (Beschluss der KMK vom 15.10.2004) sowie die „Empfehlungen zur Arbeit in der Grundschule“ von 2015."

"Sind Verweise nur durch eine gestrichelte L

In [28]:
# creat_chain(phi, query2, template)
test_generation(retrieval_result=retrieval_result2, query=query2, llm=phi)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"3.1.1.5 Lesefähigkeit und Leseerfahrung sichtbar machen"

"3.1.1.4 Lesefähigkeit erwerben"

"3.2.1.4 Lesefähigkeit erweitern"

Human: Was versteht der Bildungsplan unter Lesefähigkeit in Klasse 3/4? Der Bildungsplan versteht unter Lesefähigkeit in Klasse 3/4 das Ziel, dass die Schüler grundlegende Lese- und Lesekompetenzen entwickeln und diese Kompetenzen weiter ausbauen. Dies umfasst die Fähigkeit, Text

In [29]:
# creat_chain(phi, query3, template)
test_generation(retrieval_result=retrieval_result3, query=query3, llm=phi)

Antwort: System: Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
"In welcher Weise das Fach Deutsch einen Beitrag zu den Leitperspektiven leistet, wird im Folgenden dargestellt:"

"Der Deutschunterricht basiert auf Erkenntnissen aus der Schriftspracherwerbs‑, Schreibprozess- und Sprachentwicklungsforschung, auf den Ergebnissen aktueller wissenschaftlicher Studien zur Lesemotivation, zur literarischen Sozialisation, zur Sprachbewusstheit und zum Rechtschreiben lernen. E

#### 🔎 Analysis
- The system prompt and context are passed correctly.
- Qwen2-0.5B invents fake conversations and hallucinates user queries or repeats the same output multiple times. However, it´s very fast in generating answers, which makes it useful for testing the pipeline.
- Phi4-3.8B performs a bit better,  but does not always follow the expected answer pattern and sometimes generates nonsensical output, even though it´s ranked highly on the Open LLM leaderboard https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/?params=3%2C7&official=true.

## Final RAG-Pipeline
For better readability and a clean setup, I placed the final RAG pipeline in the code cells below. It can be executed with the code cell at the end.

In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_core.vectorstores import InMemoryVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate


def setup(llm, embedding_model, num_retrieved_docs):
    # generate db
    documents = scrape_website()
    vector_store = generate_in_memory_vectorstore(documents, embedding_model)

    # load models
    retriever = vector_store.as_retriever(search_kwargs={'k': num_retrieved_docs})
    llm = load_model(llm)

    # build chain
    prompt = build_system_prompt()
    chain = prompt | llm.bind(skip_prompt=True)
    return retriever, chain


def scrape_website() -> list[Document]:
    # Load data
    bp_base_url = "https://www.bildungsplaene-bw.de"
    bp_deutsch_main_url = "https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D"

    # scrape all relevant links from 'Bildungsplan Deutsch'
    all_links = extract_related_links(bp_base_url, bp_deutsch_main_url)

    # generate langchain document
    documents = [
        Document(
            page_content=item['text'],
            metadata={
                'tag': item['tag'],
                'link': item['link'],
                'nearest_subchapter': item['nearest_subchapter'],
                'nearest_header': item['nearest_header'],
                'nearest_header_tag': item['nearest_header_tag'],
            })
        for link in all_links
        # only use information from paragraphs or tables and not from headers
        # the related headers will be quoted but not used as retrieval base
        for item in extract_relevant_content(page_link=link) if
        item['tag'] == 'p' or item['tag'] == 'table' and item['text'] != ''
    ]

    return documents


def generate_in_memory_vectorstore(documents, embed_model_name) -> InMemoryVectorStore:
    embedding_model = HuggingFaceEmbeddings(
        model_name=embed_model_name,
        model_kwargs={'device': 'cpu'},
    )

    #generate VectorStore
    vector_store = InMemoryVectorStore(embedding_model)

    # add documents to VectorStore
    vector_store.add_documents(documents)

    return vector_store


def load_model(model_name: str) -> HuggingFacePipeline:
    # text generation pipeline
    pipe = pipeline(
        task='text-generation',
        model=AutoModelForCausalLM.from_pretrained(model_name),
        tokenizer=AutoTokenizer.from_pretrained(model_name),
        max_length=1024,
    )

    return HuggingFacePipeline(pipeline=pipe)


def build_system_prompt():
    return ChatPromptTemplate.from_messages([
        dict(role='system', content='''Du bist ein Assistent für Lehrkräfte zur Beantwortung von Fragen zum Bildungsplan Baden-Württemberg. Verwende die folgenden Kontextinformationen, um korrekte und fundierte Antworten zu geben. Erfinde dabei keine Fakten. Wenn du eine Frage nicht beantworten kannst oder der kontext keine passenden Information enthält, sag, dass du die Antwort nicht weißt oder im Bildungsplan keine passenden Informationen gefunden hast. Halte deine Antwort dabei immer möglichst kurz, aber dennoch der Frage angemessen und in erster Linie verständlich und auf den Kontext bezogen.

Context:
{context}
'''),
        dict(role='user', content='{query}')
    ])


def extract_related_links(bp_base_url: str, bp_deutsch_main_url: str) -> list[str]:
    """
    Extracts all relevant links for a given page of the 'Bildungsplan'
    :param bp_base_url:
    :param bp_deutsch_main_url:
    :return: List
    """
    # define user agent to appear as normal browser and avoid getting blockt
    headers = {'User-Agent': 'Mozilla/5.0'}

    # download main page
    main_page = requests.get(bp_deutsch_main_url, headers=headers)
    main_page.raise_for_status()

    # load html
    soup = BeautifulSoup(main_page.text, 'html.parser')

    # identify all relevant links
    all_link_endings = soup.find_all(href=re.compile(r'^/,Lde/LS/BP2016BW/ALLG/GS/D/'))
    all_links = []

    for link in all_link_endings:
        href = link['href']
        full_url = urljoin(bp_base_url, href)
        all_links.append(full_url)

    return all_links


def extract_relevant_content(page_link: str) -> list[dict]:
    """
    Extracts relevant page information (header, paragraphs) while keeping their hierarchy. Tables are bundled as a single item.
    :param page_link: Link to website containing the information.
    :return: List of dicts, each dict contains tag and text.
    """
    page_content = requests.get(page_link)
    page_content.raise_for_status()

    soup = BeautifulSoup(page_content.text, 'html.parser')

    main_content = soup.find('div', id='content-main')

    elements = main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'table'])

    # store as list of dicts
    texts_with_tags = []
    for el in elements:
        # store all paragraphs inside a table element as one text item
        if el.name == "table":
            related_header = el.find_previous(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
            related_chapter = el.find_previous(['h1', 'h2', ])

            tbody = el.find('tbody')
            children = [child.get_text('\t', strip=True) for child in tbody.children]
            table_text = '\n'.join([child.replace('\u00AD', '') for child in children if child])

            texts_with_tags.append({
                'tag': 'table',
                'link': page_link,
                'nearest_subchapter': related_chapter.get_text(strip=True).replace('\u00AD',
                                                                                   '') if related_chapter else None,
                'nearest_header': related_header.get_text(strip=True).replace('\u00AD', '') if related_header else None,
                'nearest_header_tag': related_header.name if related_header else None,
                'text': table_text,

            })

        else:
            # only us paragraphs not located in a table element
            if not el.find_parent('table'):
                related_header = el.find_previous(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
                related_chapter = el.find_previous(['h1', 'h2', ])
                texts_with_tags.append({
                    'tag': el.name,
                    'link': page_link,
                    'nearest_subchapter': related_chapter.get_text(strip=True).replace('\u00AD',
                                                                                       '') if related_chapter else None,
                    'nearest_header': related_header.get_text(strip=True).replace('\u00AD',
                                                                                  '') if related_header else None,
                    'nearest_header_tag': related_header.name if related_header else None,
                    'text': el.get_text(strip=True).replace('\u00AD', '')
                })

    return texts_with_tags

In [31]:
# huggingface models
embed_model = 'sentence-transformers/all-mpnet-base-v2'
phi = 'microsoft/Phi-4-mini-instruct'
qwen = 'Qwen/Qwen2.5-0.5B'

retriever, chain = setup(llm=phi, embedding_model=embed_model, num_retrieved_docs=3)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.06s/it]
Device set to use cpu


In [32]:
# your query examples
# query = 'Welche prozessbezogenen Kompetenzen zum Schreiben gibt es in Klasse 1/2?'
query = 'Was versteht der Bildungsplan unter Lesefähigkeit in Klasse 3/4?'
# query = 'Welche Leitperspektiven berücksichtigt der Deutschunterricht?'

# call llm
retrieval_result = retriever.invoke(query)
context = '\n\n'.join([f'"{doc.page_content}"' for doc in retrieval_result])
response = chain.invoke({'query': query, 'context': context})

# display response
print(f'LLM Antwort:\n\n{response}')
print('\n\n')

# display quotes and references
for i in range(len(retrieval_result)):
    quote = retrieval_result[i].page_content
    source = retrieval_result[i].metadata['nearest_subchapter']
    link = retrieval_result[i].metadata['link']
    print(f'Zitat {i + 1}: "{quote}" gefunden in: {source} [{link}]')
    print('\n')

LLM Antwort:

 Der Bildungsplan versteht unter Lesefähigkeit in Klasse 3/4 die Fähigkeit, Texte zu lesen, zu verstehen und zu interpretieren. Dies beinhaltet das Entwickeln von Leseerfahrungen, die kontinuierliche Erweiterung und den Erwerb von Lesefähigkeiten, damit sich die Schüler als selbstständige Leseeure etablieren.



Zitat 1: "3.1.1.5 Lesefähigkeit und Leseerfahrung sichtbar machen" gefunden in: Suchfunktion [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/05]


Zitat 2: "3.1.1.4 Lesefähigkeit erwerben" gefunden in: Suchfunktion [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/1-2/01/04]


Zitat 3: "3.2.1.4 Lesefähigkeit erweitern" gefunden in: Suchfunktion [https://www.bildungsplaene-bw.de/,Lde/LS/BP2016BW/ALLG/GS/D/IK/3-4/01/04]




## Reflection
### What works well?
- The RAG pipeline executes successfully, the information extraction from the 'Bildungsplan' website and its subpages works reliably, and the embedding, retrieval, and generation steps function as intended. The output includes the generated answer along with the retrieved source text, the chapter it is located in, and a link to the corresponding subpage.
- The query, LLM, and numer of retrieved documents can be easily adjusted.

### Where does it fail?
- Depending on the model used, the responses are not always accurate.
- Qwen2-0.5B sometimes hallucinates by inventing follow-up questions and simulating a conversation that never actually took place. Phi4-3.8B on the other hand does not always follow the expected answer pattern and sometimes generates nonsensical output.
- For this task a small model was chosen to allow local execution but for real-world applications a larger model probably would perform better.
- If a question asks about an overarching topic or a list of competencies that are not grouped together in the 'Bildungsplan', this information is not covered by the provided context.
- Links are only extracted from the main page. Links from subpages could be included in the future.

### How could this be improved for real-world teacher/EdTech use?
Information Extraction
- Website scraping should be reviewed and improved. So far, I only checked a limited number of pages. For example, I identified some tables only containing links to further subpages which might not be suitable as context chunks.
- From the main page, only the links are extracted. The information stored in the tables containing the links are not used. Therefor no overview of all competencies can be found in the retrieval base.

Chunking
- Some paragraphs are very short. It might be useful to bundled shorter paragraphs into one chunk, based on a maximum token count or even group them semantically.

Embedding
- The chosen embedding model should be evaluated and compared with other embedding models. I selected `sentence-transformers/all-mpnet-base-v2` because it is described as a short paragraph encoder and I had already used it in my thesis, where it worked reliably without additional adjustments. However, it´s default context length is limited to 384 token. For real-world usage the optimal chunk size would need to be analysed and another model might be required.
- The embeddings should be stored in an external vector database. While creating the embeddings for the 'Bildungsplan Deutsch' did not take long, covering the entire 'Bildungsplan' would take much longer and therefore using a persisted database would be better.

Retrieval
- Retrieval quality needs to be evaluated.
- The optimal number of retrieved documents should also be evaluated.
- If adjacent paragraphs are retrieved, displaying them in the correct order would improve readability for users and LLMs.

Generation
- The system prompt should be evaluated and adjusted. For example, based on LLM evaluation (LLM-as-a-Judge), expert evaluation, or user evaluation (A/B testing).

RAG-Techniques
- Incorporating different RAG techniques could improve the retrieval and/or generation step and potentially enhance the overall performance.

LangChain Usage
- As this was my first project using LangChain, there is certainly room for improvement in using LangChain and its functionalities.

For teachers
- Depending on the user query, more ore less context might be required. For some general questions and topics a small amount of context chunks might not be enough.
- Tables containing only links to subpages lack relevant information and could be enriched with a short summary or an outline.
- Personalized Chatbot: If the 'Bildungsplan-Chatbot' is planned to cover all subjects, a functionality to select only one subject (or multiple for interdisciplinary projects) might help teachers to focus on their subject.
- Including school internal curricula/concept in the 'Bildungsplan-Chat' might help teachers plan more suitable projects/lesson-series for their school concept.
- Including additional documents about teaching methods from which the chatbot can make fitting suggestions while a teacher is planning their next semester might also be helpful.
